In [1]:
import pandas as pd
import joblib

# Load scored test data
test_df = pd.read_csv(r"E:\Projects Data Scientist\credit-risk-project\data\processed\test_scored.csv")

y_test = test_df["actual_default"]
y_test_pd = test_df["pd"]

In [2]:
# Create Risk Bands
import numpy as np

def assign_risk_band(pd):
    if pd <= 0.05:
        return "Low"
    elif pd <= 0.15:
        return "Medium"
    else:
        return "High"

test_df["risk_band"] = test_df["pd"].apply(assign_risk_band)

In [3]:
# Evaluate Default Rate by Risk Band
band_summary = test_df.groupby("risk_band").agg(
    count=("actual_default", "size"),
    default_rate=("actual_default", "mean")
).reset_index()

band_summary

,risk_band,count,default_rate
0,High,39242,0.075582
1,Low,27,0.037037
2,Medium,5731,0.007154


In [4]:
# Approval Rate Impact
approval_rate = (
    test_df[test_df["risk_band"] == "Low"].shape[0]
    / test_df.shape[0]
)

print(f"Approval Rate: {approval_rate:.2%}")

Approval Rate: 0.06%


In [5]:
# KS-Driven Cutoff
ks_df = test_df[["pd", "actual_default"]].copy()
ks_df = ks_df.sort_values("pd").reset_index(drop=True)

In [6]:
ks_df["cum_bad"] = (
    ks_df["actual_default"].cumsum() /
    ks_df["actual_default"].sum()
)

ks_df["cum_good"] = (
    (1 - ks_df["actual_default"]).cumsum() /
    (1 - ks_df["actual_default"]).sum()
)

In [7]:
ks_df["ks"] = abs(ks_df["cum_bad"] - ks_df["cum_good"])

In [8]:
ks_max = ks_df["ks"].max()

ks_cutoff = ks_df.loc[
    ks_df["ks"].idxmax(), "pd"
]

print(f"Maximum KS: {ks_max:.3f}")
print(f"KS-based PD Cutoff: {ks_cutoff:.4f}")

Maximum KS: 0.505
KS-based PD Cutoff: 0.4668


In [9]:
test_df["ks_decision"] = np.where(
    test_df["pd"] <= ks_cutoff,
    "Approve",
    "Reject"
)

In [10]:
test_df

,RevolvingUtilizationOfUnsecuredLines,DebtRatio,age,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,TotalDelinquencyCount,HighUtilizationFlag,IncomeMissingFlag,actual_default,pd,risk_band,ks_decision
0,0.000000,0.087478,50,9830.0,6,0,2,0,0,1,0.225183,High,Approve
1,0.005770,0.063603,52,10486.0,16,1,0,0,0,0,0.210527,High,Approve
2,0.807706,0.555962,64,4100.0,8,2,0,1,0,0,0.591470,High,Reject
3,0.001076,0.269278,53,2450.0,8,0,0,0,0,0,0.175930,High,Approve
4,0.022680,0.369809,58,17600.0,17,2,0,0,0,1,0.199152,High,Approve
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,1.000000,14.000000,46,5400.0,0,0,5,1,1,1,0.868309,High,Reject
44996,0.065615,1292.000000,48,5400.0,7,2,1,0,1,0,0.260321,High,Approve
44997,1.000000,0.059608,51,7800.0,3,0,1,1,0,0,0.705881,High,Reject
44998,0.020657,43.000000,67,5400.0,7,0,0,0,1,0,0.139051,Medium,Approve
